<a href="https://colab.research.google.com/github/pilipandr770/GOIT_diplom/blob/main/GOIT_Diplom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Крок 1: Імпорт необхідних бібліотек
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib
import json

In [3]:
# Крок 2: Завантаження даних з локального файлу
from google.colab import files
uploaded = files.upload()

Saving archive (3).zip to archive (3).zip


In [4]:
# Крок 3: Визначення шляху до завантаженого файлу
file_name = list(uploaded.keys())[0]
data = pd.read_csv(file_name)

In [5]:
# Крок 4: Перегляд перших рядків даних
data.head()

,id,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,reamining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn
0,15,1,0,11.95,25,0.14,0,8.4,2.3,0,0
1,18,0,0,8.22,0,NaN,0,0.0,0.0,0,1
2,23,1,0,8.91,16,0.00,0,13.7,0.9,0,1
3,27,0,0,6.87,21,NaN,1,0.0,0.0,0,1
4,34,0,0,6.39,0,NaN,0,0.0,0.0,0,1


In [6]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=data)

https://docs.google.com/spreadsheets/d/1PDHhHHf8s_ckEeTWynNKOzdiiumqU4c9EEgV9WDoXNM#gid=0


/usr/local/lib/python3.10/dist-packages/google/colab/sheets.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return frame.applymap(_clean_val).replace({np.nan: None})


In [7]:
# Крок 5: Видалення дублікатів
# Видаляємо всі дублікати, якщо такі є
data = data.drop_duplicates()

In [8]:
# Крок 6: Видалення колонки ID, якщо така є
# Видаляємо колонку 'ID' або 'customerID', якщо вона присутня
data = data.drop(columns=['ID', 'customerID'], errors='ignore')

In [9]:
# Крок 7: Обробка відсутніх значень
# Заповнюємо відсутні значення медіаною для числових колонок
data = data.fillna(data.median())

In [10]:
# Крок 8: Перейменування колонки 'churn' для уніфікації
# Перейменовуємо колонку 'churn' в 'Churn', якщо вона присутня
if 'churn' in data.columns:
    data.rename(columns={'churn': 'Churn'}, inplace=True)

In [11]:
# Крок 9: Кодування категоріальних змінних
# Перетворюємо категоріальні змінні в числові, використовуючи get_dummies
data = pd.get_dummies(data, drop_first=True)

In [12]:
# Крок 10: Поділ на ознаки та цільову змінну
# Виділяємо ознаки (X) та цільову змінну (y)
X = data.drop(columns=['Churn'])
y = data['Churn']

In [13]:
# Крок 11: Поділ на навчальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Крок 12: Створення та навчання моделей
models = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
}

metrics = {}

In [17]:
# Крок 14: Збереження метрик оцінки моделей
with open('/content/model_metrics.json', 'w') as f:
    json.dump(metrics, f) # Indented this line to be within the 'with' block

In [20]:
# Крок 13: Навчання моделі та оцінка її якості
for name, model in models.items():
    # Навчання моделі
    model.fit(X_train, y_train)

    # Прогнозування на тестовій вибірці
    y_pred = model.predict(X_test)

    # Оцінка моделі
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    metrics[name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

    # Виведення метрик моделі
    print(f'Model: {name}')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1}')
    print('------------------------')

    # Збереження моделі - This line was incorrectly indented before.
    # Now it's correctly placed within the loop.
    joblib.dump(model, f'/content/{name}_model.pkl')

Model: RandomForest
Accuracy: 0.9696990660671048
Precision: 0.9780937740199847
Recall: 0.9662110858010631
F1 Score: 0.9721161191749427
------------------------


In [21]:

# Крок 14: Збереження метрик оцінки моделей
with open('/content/model_metrics.json', 'w') as f:
    json.dump(metrics, f)

# Крок 15: Збереження назв ознак після підготовки X
feature_names = X.columns.tolist()
with open('/content/feature_names.json', 'w') as f:
    json.dump(feature_names, f)

# Крок 16: Збереження навчальних та тестових наборів даних
X_train.to_csv('/content/X_train.csv', index=False)
X_test.to_csv('/content/X_test.csv', index=False)
y_train.to_json('/content/y_train.json')
y_test.to_json('/content/y_test.json')